In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [884]:
# 감성사전 txt 불러오기
def load_sentiment_dictionary(file_path):
    sentiment_dict = set()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            word = line.strip()  # 각 줄의 공백 및 개행문자 제거
            sentiment_dict.add(word)
    return sentiment_dict

# CSV 파일을 데이터프레임으로 변환
def load_csv_file(csv_file):
    df = pd.read_csv(csv_file, encoding='utf-8')
    return df

# 감성사전과 토큰을 비교하여 부정어인 경우 'neg_word' 열에 추가
def add_word_column(df, sentiment_dict):
    word_column = []
    for token in df['token']:
        words = token.split()
        positive_words = [word for word in words if word in sentiment_dict]
        word_column.append(' '.join(positive_words))
    df['neg_word'] = word_column
    return df

# 데이터프레임을 csv 파일로 저장
def save_dataframe_to_csv(df, output_file):
    df.to_csv(output_file, index=False, encoding='utf-8')

In [885]:
# 감성사전 텍스트 파일 불러오기
sentiment_dict = load_sentiment_dictionary('text/neg_word.txt')

# csv 파일 불러오기
df = load_csv_file('data/충청남도_neg.csv')

In [886]:
# 감성사전과 토큰을 비교하여 부정어인 경우 'neg_word' 열에 추가
df_with_words = add_word_column(df, sentiment_dict)

In [887]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_with_words["neg_word"])

feature_names = vectorizer.get_feature_names_out()

# TF-IDF 값이 가장 큰 단어 추출
def get_top_keywords(tfidf_matrix, feature_names, top_n=20):
    keywords = []
    for i in range(tfidf_matrix.shape[0]):
        scores = {feature_names[col]: tfidf_matrix[i, col] for col in tfidf_matrix[i].nonzero()[1]}
        top_keywords = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
        keywords.append([kw for kw, _ in top_keywords])
    return keywords

# 상위 20개의 핵심 키워드 추출
top_keywords_per_review = get_top_keywords(tfidf_matrix, feature_names, top_n=20)

df_with_words['neg_keywords'] = [' '.join(kw_list) for kw_list in top_keywords_per_review]

In [888]:
df_with_words

,시도,시군구,code,title,content,full_content,token,Sentiment_Label,neg_word,neg_keywords
0,충청남도,계룡시,44250.0,<b>여행</b> 72. <b>충청남도 여행</b> 첫 번째 이야기(1일차 결과 &...,기간: 2022.6.6.(월)09:10~7(화)20:00[1박2일의 여정 중 제1일...,<b>여행</b> 72. <b>충청남도 여행</b> 첫 번째 이야기(1일차 결과 &...,첫 이야기 일차 결과 후기 기간 월화 박일 여정 일차 목적지 남부 지역 전시 특이 ...,Negative,늦다 늦다 싸우다 슬프다 훼손 아쉽다 당하다 슬프다 지다 지다 아쉬움 아쉽다 병 병 짐,슬프다 늦다 아쉽다 훼손 아쉬움 당하다 지다 싸우다
1,충청남도,계룡시,44250.0,<b>여행</b> 69. <b>충청남도 여행</b> 1차(계획 짜기),기간: 2022.6.6(월)08:00~7(화)19:00[1박2일]목적지: 충청남도 ...,<b>여행</b> 69. <b>충청남도 여행</b> 1차(계획 짜기)기간: 2022...,차 계획 짜다 기간 월화 박일 목적지 남부 지역 전시 특이 사항 은퇴 부부 차 역사...,Negative,병 화,
2,충청남도,계룡시,44250.0,논산 <b>여행</b> 고정리 김국광 사당과 양천허씨정려,광산김씨 집성촌 논산시 연산면 고정리 탐방기논산 여행 고정리 김국광 사당과 양천허씨...,논산 <b>여행</b> 고정리 김국광 사당과 양천허씨정려광산김씨 집성촌 논산시 연산...,정리 김국광 사당 양천허씨 정려 광산김씨 집성촌 연산면 정리 탐방 정리 김국광 사당...,Negative,죽다 버리다 버리다 죽다 죽다 어색하다 버리다 어이없다 지다 엉터리 엉터리 지다 지...,죽다 지다 걱정 엉터리 불쌍하다 몰래 버리다 잃다 어렵다 떨어지다 의심 다투다 추위...
3,충청남도,계룡시,44250.0,[완벽정리] <b>충청남도</b> 태양광발전사업 허가 기준 완벽 정리 (ft....,태양광발전사업 준비를 하시는 예비사업자 분이라면 여러 커뮤니티를 통해 토지를 알아보...,[완벽정리] <b>충청남도</b> 태양광발전사업 허가 기준 완벽 정리 (ft.......,완벽 정리 태양광 발전 사업 허가 기준 완벽 정리 ft 태양광 발전 사업 준비 예비...,Negative,어렵다 지다,어렵다 지다
4,충청남도,계룡시,44250.0,<b>충청남도</b> 서남부에 위치한 서천군의 대표(?) 기차역 - 서천역,충청남도 서남부에 위치한 서천군은 충청남도에서 면적이 계룡시 다음으로 작고 인구가 ...,<b>충청남도</b> 서남부에 위치한 서천군의 대표(?) 기차역 - 서천역충청남도 ...,남부 위치 대표 기차역 서천역 남부 위치 면적 작고 인구 점점 감소 추세 가깝다 생...,Negative,모시 모시 떨어지다 아깝다 버리다 화,모시 아깝다 버리다 떨어지다
...,...,...,...,...,...,...,...,...,...,...
192,충청남도,홍성군,44800.0,[<b>충청남도</b>] 가보고 싶은 캠핑장 추천 리스트,사업장명\n소재지전화\n도로명전체주소\n문화체육업종명\n보험기관명\n청포대휴야영장\...,[<b>충청남도</b>] 가보고 싶은 캠핑장 추천 리스트사업장명\n소재지전화\n도로...,캠핑장 추천 리스트 사업 명소 재다 전화 도로명 소문 체육 업종 보험 기관 청포 야...,Negative,손해 벌 욕 밉다 지다 죽다 손해 손해 벌,손해 죽다 밉다 지다
193,충청남도,홍성군,44800.0,<b>충청남도</b> 홍성여행 (<b>충청</b>도의 첫 순교터 홍주성지 : 순교터...,홍성의 역사적인 주요관광지와 볼거리는 홍주읍성에 집중되어 있는데홍주읍성은 천주교 순...,<b>충청남도</b> 홍성여행 (<b>충청</b>도의 첫 순교터 홍주성지 : 순교터...,첫 순교 홍주성 순교 역사 관광지 볼거리 홍주읍성 집중 홍주읍성 천주교 순교 성지 ...,Negative,심하다 죽다 굶다 죽다 아프다 아프다 당하다 흘리다 아픔 파괴 없어지다 당하다 부족...,당하다 아프다 파괴 죽다 굶다 없어지다 아픔 부족하다 심하다 흘리다 버리다
194,충청남도,홍성군,44800.0,[<b>충청남도</b> <b>홍성군</b> <b>맛집</b>] 병천토종순대 홍성점,안녕하세요.오늘은 홍성군에 위치한‘병첨토종순대’를 리뷰해보려고 합니다.​수업 끝나고...,[<b>충청남도</b> <b>홍성군</b> <b>맛집</b>] 병천토종순대 홍성점안...,병천 토종 순대 안녕하다 오늘 위치 병 토종 순대 리뷰 해보다 수업 끝나다 밥 먹다...,Negative,병 아쉽다 아쉽다 아쉽다 아쉬움 심하다 아쉬움,아쉽다 아쉬움 심하다
195,충청남도,홍성군,44800.0,<b>충청남도</b> <b>홍성군</b> 상가 정보... 창업한 예비 <b>맛집</...,​충남 홍성군에2020년 3분기(7월~9월)에 창업한 음식점 상가 정보입니다.​음식...,<b>충청남도</b> <b>홍성군</b> 상가 정보... 창업한 예비 <b>맛집</...,상가 정보 창업 예비 상가 현황 분기 월월 창업 음식점 상가 정보 음식점 식당 뿐 ...,Negative,밉다,밉다


In [889]:
result_df = df_with_words.groupby(['시도','시군구','code'])['neg_keywords'].agg(lambda x: ' '.join(map(str, x))).reset_index()

In [ ]:
output_file_path = 'data/충청남도_neg_keywords.csv'
save_dataframe_to_csv(result_df, output_file_path)

In [891]:
data = pd.read_csv('data/충청남도_neg_keywords.csv')

In [892]:
data

,시도,시군구,code,neg_keywords
0,충청남도,계룡시,44250.0,슬프다 늦다 아쉽다 훼손 아쉬움 당하다 지다 싸우다 죽다 지다 걱정 엉터리 불쌍하...
1,충청남도,공주시,44150.0,막히다 버리다 실수 위험하다 잘못 피로 답답하다 우울하다 스트레스 흐리다 벗어나다 ...
2,충청남도,금산군,44710.0,슬프다 늦다 아쉽다 훼손 아쉬움 당하다 지다 싸우다 싸움 싸우다 모시 몰래 훼손...
3,충청남도,논산시,44230.0,떨어지다 지다 덥다 싫다 틀어지다 부서지다 불편하다 영정 싸우다 패배하다 위기 불...
4,충청남도,당진시,44270.0,떨어지다 수선 지다 쓸쓸하다 망설이다 안쓰럽다 부러지다 느리다 일부러 모시 떨어지...
5,충청남도,보령시,44180.0,지루하다 속상하다 비싸다 힘들다 부족하다 지나치다 밉다 느끼하다 아쉽다 부상 무섭...
6,충청남도,부여군,44760.0,밉다 지다 어수선하다 흉물 해치다 허전하다 헤매다 싫다 잃다 욕심 쓸데없다 짧다 ...
7,충청남도,서산시,44210.0,밉다 부러지다 아픔 욕심 덥다 아쉽다 버리다 지다 치욕스럽다 죄수 잘못 당하다 모...
8,충청남도,서천군,44770.0,모시 밉다 시들다 막히다 떨어지다 밉다 복잡하다 오래되다 아프다 모시 놓치다 오래...
9,충청남도,아산시,44200.0,짧다 부자연스럽다 움츠리다 불신 약하다 위축 나무라다 없애다 공포 오래되다 지다 ...


In [893]:
# 여러 개의 CSV 파일 읽기
files = ['data/강원특별자치도_neg_keywords.csv', 'data/경기도_neg_keywords.csv', 'data/경상남도_neg_keywords.csv', 'data/경상북도_neg_keywords.csv', 'data/광주광역시_neg_keywords.csv', 'data/대구광역시_neg_keywords.csv', 'data/대전광역시_neg_keywords.csv', 'data/부산광역시_neg_keywords.csv','data/서울특별시_neg_keywords.csv', 'data/세종특별자치시_neg_keywords.csv','data/울산광역시_neg_keywords.csv','data/인천광역시_neg_keywords.csv','data/전라남도_neg_keywords.csv','data/전라북도_neg_keywords.csv','data/제주특별자치도_neg_keywords.csv','data/충청남도_neg_keywords.csv','data/충청북도_neg_keywords.csv']

# 데이터프레임들을 수직으로 합치기
dfs = [pd.read_csv(file) for file in files]
neg_word_all = pd.concat(dfs, ignore_index=True)

In [894]:
neg_word_all

,시도,시군구,code,neg_keywords
0,강원특별자치도,강릉시,42150.0,부상 당하다 다치다 화가 짠하다 망상 모시 태우다 버리다 피해 떨어지다 위험하다 ...
1,강원특별자치도,고성군,42820.0,지다 홀로 짧다 아쉽다 요란하다 고집 까칠하다 떨구다 거슬리다 추위 복잡하다 고작 ...
2,강원특별자치도,동해시,42170.0,망상 성나다 피해 조심하다 안타깝다 아쉽다 지다 망상 싫다 떨어지다 아찔하다 흐리...
3,강원특별자치도,삼척시,42230.0,같잖다 싫어하다 잘못 조심하다 무섭다 무너지다 약하다 흘리다 한숨 욕심 늦다 비싸다...
4,강원특별자치도,속초시,42210.0,아프다 지경 소음 심하다 답답하다 어리석다 무섭다 부담
...,...,...,...,...
244,충청북도,청주시 상당구,43111.0,막막하다 낯설다 부족 부족하다 떨어지다 어렵다 싫다 막히다 부족 죽다 싸우다 어렵다...
245,충청북도,청주시 서원구,43112.0,어렵다 지다 막막하다 낯설다 부족 부족하다 떨어지다 어렵다 요란하다 무섭다 포기 싸...
246,충청북도,청주시 청원구,43114.0,막막하다 낯설다 부족 부족하다 떨어지다 어렵다 아쉽다 짧다 불편하다 아쉬움 아깝다...
247,충청북도,청주시 흥덕구,43113.0,죽다 싸우다 어렵다 두렵다 분하다 지다 버리다 아쉽다 막막하다 낯설다 부족 부족하다...


In [895]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(neg_word_all["neg_keywords"])

feature_names = vectorizer.get_feature_names_out()

# TF-IDF 값이 가장 큰 단어 추출
def get_top_keywords(tfidf_matrix, feature_names, top_n=5):
    keywords = []
    for i in range(tfidf_matrix.shape[0]):
        scores = {feature_names[col]: tfidf_matrix[i, col] for col in tfidf_matrix[i].nonzero()[1]}
        top_keywords = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
        keywords.append([kw for kw, _ in top_keywords])
    return keywords

# 상위 5개의 핵심 키워드 추출
top_keywords_per_review = get_top_keywords(tfidf_matrix, feature_names, top_n=5)

neg_word_all['neg_top_keywords'] = [' '.join(kw_list) for kw_list in top_keywords_per_review]

In [896]:
neg_word_all = neg_word_all.groupby(['시도','시군구','code'])['neg_top_keywords'].agg(lambda x: ' '.join(map(str, x))).reset_index()

In [897]:
neg_word_all

,시도,시군구,code,neg_top_keywords
0,강원특별자치도,강릉시,42150.0,분풀이 불행하다 태우다 살벌하다 억지로
1,강원특별자치도,고성군,42820.0,망하다 늦어지다 쓰리다 고작 까칠하다
2,강원특별자치도,동해시,42170.0,망상 무섭다 아찔하다 부담 성나다
3,강원특별자치도,삼척시,42230.0,지지 소음 같잖다 조심하다 나쁘다
4,강원특별자치도,속초시,42210.0,어리석다 소음 지경 답답하다 부담
...,...,...,...,...
244,충청북도,청주시 상당구,43111.0,부족 아쉽다 막막하다 박약 두렵다
245,충청북도,청주시 서원구,43112.0,고집 지다 여리다 화가 처절하다
246,충청북도,청주시 청원구,43114.0,어렵다 서럽다 싸우다 아쉽다 치욕
247,충청북도,청주시 흥덕구,43113.0,무모하다 요란하다 싸우다 방정 너무하다


In [898]:
neg_word_all.to_csv('data/neg_top_keywords.csv', index=False, encoding='utf-8-sig')